In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("Churn_Modelling.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
df['Geography'].value_counts()

France     5014
Germany    2509
Spain      2477
Name: Geography, dtype: int64

In [4]:
X = df.iloc[:,3:13]

y = df.iloc[:,13]

In [5]:
# Let's do one hot encoding with these categorical column

geography = pd.get_dummies(X['Geography'],drop_first=True)

gender = pd.get_dummies(X['Gender'],drop_first=True)

In [6]:
X = pd.concat([X,geography,gender],axis=1)

In [7]:
X = X.drop(['Geography','Gender'],axis=1)

In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [9]:
from sklearn.preprocessing import StandardScaler
scalar = StandardScaler()
X_train = scalar.fit_transform(X_train)
X_test = scalar.transform(X_test)

In [10]:
print(X_train.shape)
print(y_train.shape)

(8000, 11)
(8000,)


In [20]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense

In [21]:
model = Sequential()

In [13]:
# # adding the input layer and first hidden layer
# model.add(Dense(units=8, kernel_initializer='he_uniform',activation='relu',input_dim=10))

# # Adding the second hidden layer
# model.add(Dense(units=16, kernel_initializer='he_uniform',activation='relu'))

# # Adding the third hidden layer
# model.add(Dense(units=14, kernel_initializer='he_uniform',activation="relu"))

# # Adding the output layer
# model.add(Dense(units=1, kernel_initializer='glorot_uniform',activation='sigmoid'))

# # Compiling the ANN
# model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

# model_history = model.fit(X_train,y_train,validation_split=0.33,batch_size=10,epochs=5)

## Now we do hyperparameter tuning here using GridSearchCV

In [17]:
import tensorflow as tf
from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.layers import LeakyReLU,Dense,Activation,Embedding,Flatten
from tensorflow.keras.layers import BatchNormalization
from keras.activations import relu, sigmoid
from sklearn.model_selection import GridSearchCV

In [22]:
def create_model(layers, activation):
    model = Sequential()
    for i, nodes in enumerate(layers):
        if i==0:
            model.add(Dense(nodes,input_dim=X_train.shape[1]))
            model.add(Activation(activation))
        else:
            model.add(Dense(nodes))
            model.add(Activation(activation))
    model.add(Dense(1)) # Note: no activation beyond this point
    
    model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
    return model
    
model = KerasClassifier(build_fn=create_model, verbose=0)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_22876\3582075148.py:15: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, verbose=0)


In [23]:
layers = [[20], [40, 20], [45, 30, 15]]
activations = ['sigmoid', 'relu']
param_grid = dict(layers=layers, activation=activations, batch_size = [128, 256], epochs=[30])
grid = GridSearchCV(estimator=model, param_grid=param_grid)

In [24]:
grid_result = grid.fit(X_train,y_train)

In [30]:
y_pred = grid.predict(X_test)
pred_y = (y_pred>0.5)

63/63 [==============================] - 1s 3ms/step


In [32]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [34]:
print(confusion_matrix(pred_y, y_test))
print(accuracy_score(y_pred, y_test))

[[1519  212]
 [  76  193]]
0.856
